In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

    0 = spontanic
    1 = sync
    2 = alone

In [236]:
def encode(path, num):
    df = pd.read_csv(path).iloc[8:,:]
      
    for col in df.columns:
        col1 = col.replace(" ", "_")
        df = df.rename({col: col1}, axis=1)
        
    df = df.rename({'_#_hands': 'hands_num'}, axis=1)  
     
    if(num != 2):
        df = df[df.hands_num == 2]
        df = df.sort_values(by=['_Hand_Type'])
        left, right =split(df)
    else:
        left = df[df.hands_num == 1]
        right = HandRight.iloc[8:,:]
        
        if len(left)>len(right):
            left = left.iloc[:len(right),:]
        else:
            right = right.iloc[:len(left),:]
        
    left = rename(left, 'L')
    right = rename(right, 'R')
    df = pd.concat([left, right], axis=1)
    df = df.drop(['LTime', 'RTime', 'L_Frame_ID', 'R_Frame_ID', 'Lhands_num', 'Rhands_num'], axis=1)    
    df['label']=num
    df = df[df.index % 5 == 0].reset_index(drop=True)
    
    return df

In [223]:
def split(df) :
    half = int(len(df)/2)
    hd = df.head(half)
    tl = df.tail(len(df)-half)
    hd = hd.reset_index(drop=True)
    tl = tl.reset_index(drop=True)
    return hd, tl


In [235]:
def rename(df, char):
    for col in df.columns:
        col1 = char+col
        df = df.rename({col: col1}, axis=1)
    return df

In [225]:
HandRight = pd.read_csv(r'HandRight.csv').iloc[8:,:]
for col in HandRight.columns:
    col1 = col.replace(" ", "_")
    HandRight = HandRight.rename({col: col1}, axis=1)
HandRight = HandRight.rename({'_#_hands': 'hands_num'}, axis=1) 
HandRight = HandRight[HandRight._Hand_Type == 'right']
HandRight = HandRight.dropna().reset_index(drop=True)

In [226]:
def combine(df,numOfRows):
    
    for i in range(1, numOfRows+1):
        line = ["{}{}".format(ind,'_'+str(i)) for ind in df.columns]
        if i==1:
            lineTotal = line
            lineArr = [line]
        else:
            lineTotal = lineTotal + line
            lineArr.append(line)

    headlines = pd.DataFrame(columns = lineTotal)

    
    for i in range(0, len(df)-len(df)%numOfRows, numOfRows):
    
        for j in range(0, numOfRows):
            listj = df.loc[i+j]
            listj.index = lineArr[j]
            if j==0:
                arr = [listj]
            else:
                arr.append(listj)
    
        jumpRows = pd.concat(arr)
        headlines = headlines.append(jumpRows, ignore_index = True)
        
    return headlines

In [237]:
#evyatar
ev_spo = encode(r'Evyatar636771052727603804Spontan.csv',0)
ev_sync = encode(r'Evyatar636771053639929594Sync.csv',1)
ev_al = encode(r'Evyatar636771054555711409Alone.csv',2)

In [240]:
ev_al

,L_Hand_Type,L_Position_X,L_Position_Y,L_Position_Z,L_Velocity_X,L_Velocity_Y,L_Velocity_Z,L_Pitch,L_Roll,L_Yaw,...,R_Wrist_Pos_X,R_Wrist_Pos_Y,R_Wrist_Pos_Z,R_Elbow_pos_X,R_Elbow_Pos_Y,R_Elbow_Pos_Z,R_Grab_Strenth,R_Grab_Angle,R_Pinch_Strength,label
0,left,-67.05989,132.6374,43.07516,242.26170,-100.21330,20.354860,0.971804,0.273967,1.379396,...,51.442050,207.56790,83.84519,205.9019,52.08065,240.1409,0.0,0.757436,0.194288,2
1,left,-48.03088,134.0193,45.00487,247.57070,11.43652,16.648890,0.979900,0.452308,1.268474,...,50.353630,204.64430,85.59027,208.4467,47.60641,236.6033,0.0,0.542423,0.000000,2
2,left,-23.62660,145.6105,45.24163,361.30030,233.29100,-130.042800,1.012865,0.786930,1.074275,...,49.262310,202.63750,87.44856,210.9805,41.77467,230.3715,0.0,0.326198,0.000000,2
3,left,-13.23966,155.5998,37.23442,-13.45813,130.84090,-6.762465,0.995325,1.090888,0.804033,...,49.928720,203.42260,89.70747,208.7383,38.14925,230.9063,0.0,0.125118,0.000000,2
4,left,-13.28533,167.2970,24.48908,-51.34582,215.67230,-169.914900,0.908828,1.232877,0.571521,...,48.388020,211.45620,87.61926,214.8508,57.29598,232.5951,0.0,0.098050,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,left,-79.48016,195.1782,42.76726,-49.01356,-371.70830,-132.028000,1.761022,0.991374,1.848988,...,33.900730,88.02253,105.61640,104.3432,41.91928,364.1944,0.0,1.117191,0.000000,2
864,left,-78.71684,166.9893,30.81404,-72.30141,-350.63030,-169.077300,1.851159,0.466109,1.704703,...,43.152610,92.22693,106.83550,124.4622,36.45644,360.2692,0.0,0.998979,0.000000,2
865,left,-85.85686,152.7076,20.96723,-71.34928,-33.11991,-97.190900,1.950572,0.389266,1.723750,...,28.684420,117.55710,162.31460,156.9064,18.79855,380.8957,0.0,0.310348,0.000000,2
866,left,-103.94230,166.6207,19.63715,-226.45970,252.87910,59.841460,1.897608,0.626088,1.809625,...,9.693019,119.23420,247.28070,164.2329,-38.65720,406.0116,0.0,0.700867,0.000000,2


In [242]:
#nofar
nofar_spo = encode(r'Nofar636759795182793299Spontan.csv',0)
nofar_sync = encode(r'Nofar636759797397919664Sync.csv',1)
nofar_al = encode(r'Nofar636759796290435160Alone.csv',2)

In [134]:
#oriya
oriya_spo = encode(r'Oriya636759804404113837Spontan.csv',0)
oriya_sync = encode(r'Oriya636759806131350399Sync.csv',1)
oriya_al = encode(r'Oriya636759805268396661Alone.csv',2)


In [135]:
#orya
orya_spo = encode(r'OryaB636771083605535985Spontan.csv',0)
orya_sync = encode(r'OryaB636771084494502229Sync.csv',1)
orya_al = encode(r'OryaB636771082736601528Alone.csv',2)


In [136]:
#revital
rev_spo = encode(r'Revital636770957038873193Spontan.csv',0)
rev_sync = encode(r'Revital636770957894370062Sync.csv',1)
rev_al = encode(r'Revital636770955999435879Alone.csv',2)


In [137]:
#sachar
sachar_spo = encode(r'Sachar636771045591258071Spontan.csv',0)
sachar_sync = encode(r'Sachar636771046527112972Sync.csv',1)
sachar_al = encode(r'Sachar636771047408504303Alone.csv',2)


In [138]:
#shelly
shelly_spo = encode(r'Shelly636759787508979097Spontan.csv',0)
shelly_sync = encode(r'Shelly636759789526313406Sync.csv',1)
shelly_al = encode(r'Shelly636759788383205274Alone.csv',2)


In [139]:
#yael
yael_spo = encode(r'Yael636771059544754823Spontan.csv',0)
yael_sync = encode(r'Yael636771060699529085Sync.csv',1)
yael_al = encode(r'Yael636771061659067649Alone.csv',2)


In [191]:
#yoel
yoel_spo = encode(r'Yoel636760921663075813Spontan.csv',0)
yoel_sync = encode(r'Yoel636760923490742912Sync.csv',1)
yoel_al = encode(r'Yoel636760922573364655Alone.csv',2)

In [100]:
dataFrames = [ev_spo, nofar_spo, oriya_spo, orya_spo,rev_spo, sachar_spo, shelly_spo, yael_spo, yoel_spo,
         ev_sync, nofar_sync, oriya_sync, orya_sync,rev_sync, sachar_sync, shelly_sync, yael_sync, yoel_sync,
         ev_al, nofar_al, oriya_al, orya_al,rev_al, sachar_al, shelly_al, yael_al, yoel_al ]

df = pd.concat(dataFrames)

In [101]:
df

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,label
7,336.0481,128562,left,2,-95.37997,156.0973,-40.18840,-3.056150,203.812100,19.494730,...,-148.3562,118.3093,-10.91977,-314.2976,-49.04110,81.41824,0.000000,0.000000,0.000000,0
8,336.0651,128564,right,2,132.46290,216.8003,-25.45043,98.732320,-65.536460,-218.720100,...,179.5375,165.1393,-15.53189,271.5673,-37.88867,104.38140,0.000000,0.728858,0.000000,0
9,336.0651,128564,left,2,-97.10074,161.7751,-40.36338,-98.927470,316.645200,-7.328697,...,-149.2292,121.7602,-12.78066,-299.9106,-56.55378,85.03669,0.000000,0.000000,0.000000,0
10,336.0816,128566,right,2,134.08500,214.4446,-28.93817,138.872500,-93.848970,-209.651200,...,181.0919,163.4637,-15.32005,268.2274,-43.06868,102.24200,0.000000,0.802890,0.000000,0
11,336.0816,128566,left,2,-98.59742,166.4238,-40.22689,-40.499030,197.513500,28.777420,...,-150.8117,125.7308,-14.01807,-293.6814,-57.60085,86.21560,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,764.4578,59984,left,1,-125.81370,214.8894,20.59324,20.170870,19.997410,37.082170,...,-146.6314,148.2971,44.40766,-258.0887,-77.52971,112.97060,0.116068,1.901876,0.000000,2
2246,764.4913,59986,left,1,-125.92320,215.6769,22.09264,-6.867068,21.957800,42.485630,...,-146.1113,148.8045,45.68930,-253.6847,-80.13625,109.99670,0.134497,1.952747,0.000000,2
2247,764.5253,59988,left,1,-125.62170,215.7428,23.39750,17.454730,-3.217659,31.446690,...,-144.8174,148.4588,46.66075,-249.6531,-83.33176,105.00430,0.120680,1.984825,0.000000,2
2248,764.5587,59989,left,1,-125.13520,215.6353,23.80248,25.567410,-5.648659,21.284490,...,-143.8482,148.1024,46.72861,-248.2768,-84.26936,103.46830,0.097735,1.988376,0.000000,2


In [219]:
HandRight1 = pd.read_csv(r'HandRight.csv').iloc[8:,:]
for col in HandRight1.columns:
    col1 = col.replace(" ", "_")
    HandRight1 = HandRight.rename({col: col1}, axis=1)
HandRight1 = HandRight1.rename({'_#_hands': 'hands_num'}, axis=1) 
HandRight1 = HandRight1[HandRight1._Hand_Type == 'right']
HandRight1 = HandRight1.dropna().reset_index(drop=True)

In [220]:
HandRight1 = pd.concat([HandRight1,HandRight1])

,Time,_Frame_ID,_Hand_Type,hands_num,_Position_X,_Position_Y,_Position_Z,_Velocity_X,_Velocity_Y,_Velocity_Z,...,_Yaw,_Wrist_Pos_X,_Wrist_Pos_Y,_Wrist_Pos_Z,_Elbow_pos_X,_Elbow_Pos_Y,_Elbow_Pos_Z,_Grab_Strenth,_Grab_Angle,_Pinch_Strength
0,128.4214,15494,right,1,18.405520,220.4556,17.57367,0.046715,78.36597,-32.120880,...,-0.277499,49.438360,201.1063,83.36790,199.6805,32.06783,229.3652,0.0,0.305218,0.0
1,128.4380,15496,right,1,18.293360,222.1953,17.22853,-10.397290,97.65856,-15.440830,...,-0.288245,49.790550,201.8538,82.53721,199.1181,32.01506,228.5434,0.0,0.351661,0.0
2,128.4547,15498,right,1,18.205560,223.6056,17.36227,0.300404,74.32468,22.264400,...,-0.300449,50.220530,202.4105,82.17856,198.7624,32.08682,228.4237,0.0,0.466381,0.0
3,128.4712,15500,right,1,18.341280,224.4987,17.81224,9.184148,49.51511,22.582000,...,-0.310815,50.765820,202.7484,82.26896,199.3933,33.14685,229.2643,0.0,0.599450,0.0
4,128.4880,15502,right,1,18.410930,225.3138,17.95707,3.805511,38.83008,7.956595,...,-0.318157,51.162460,203.4144,82.24403,200.2325,36.39592,231.7279,0.0,0.732624,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4341,201.0806,23867,right,1,-8.623579,163.7605,207.30110,-271.901400,-12.95597,1133.339000,...,0.004875,5.541947,119.3280,267.78980,171.7839,-42.17188,410.2199,0.0,0.765803,0.0
4342,201.0967,23869,right,1,-13.097740,162.5545,225.69850,-266.212900,-102.12860,1029.193000,...,0.003377,1.698353,117.0805,285.09920,179.2368,-44.11964,413.5244,0.0,0.783391,0.0
4343,201.1133,23871,right,1,-16.722380,160.9650,240.58440,-192.594300,-87.53856,769.651500,...,0.004909,-1.484266,114.0260,298.58760,186.0559,-44.35751,415.7693,0.0,0.684677,0.0
4344,201.1299,23873,right,1,-19.684070,160.7849,251.09980,-158.997200,73.45740,557.203700,...,0.004470,-4.015731,112.9914,308.15980,190.9921,-42.38844,416.7549,0.0,0.591905,0.0
